# Import packages

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math
from src.langsim_utils import *

ModuleNotFoundError: No module named 'src.language_similarity_utils'

# Data cleaning

#### Scriptdata

In [5]:
# add the language name and code of the above row to the rows with missing values
for index, row in scriptdata.iterrows():
    if pd.isna(row['Language']):
        scriptdata.at[index, 'Language'] = scriptdata.at[index - 1, 'Language']
        scriptdata.at[index, 'Code'] = scriptdata.at[index - 1, 'Code']

scriptdata = scriptdata.drop(columns=['ML','P','Code.1','MS']) # drop the unnecessary columns
scriptdata = scriptdata.groupby(['Language', 'Code'])['Script'].agg(', '.join).reset_index() # group the languages by name and add the different script types

scriptdata.rename(columns = {'Language':'Name'}, inplace = True) 

# Change names that differ in the nllb and wals naming code?
scriptdata.loc[scriptdata['Name'] == 'Bangla', 'Name'] = 'Bengali'
scriptdata.loc[scriptdata['Name'] == 'Chinese', 'Name'] = 'Mandarin'

#### Merge scriptdata with walsdata

In [6]:
walsdata = pd.merge(walsdata, scriptdata)

#### Select relevant languages

In [ ]:
walsdata_test = walsdata.loc[walsdata['wals_code'].isin(['eng','hin','nep','ben','tel', 'tml', 'cnt','kor','mnd','jpn','vie','lao','tha','khm','brm','mly','ind','bal','phi','nld','afr','fri','ger','nor','fin','dsh','ctl','por','rus','ukr','pol','slo','scr','cze','hun','bul','swa','zul'])]
# walsdata_mgsm = walsdata.loc[walsdata['wals_code'].isin(['eng','ben','mnd','fre','ger','jpn','rus','spa','swa','tel','tha'])]
# walsdata_xcopa = walsdata.loc[walsdata['wals_code'].isin(['eng','est','hcr','ind','que','tml','tur','vie','mnd','swa','tha'])]
# languages_llama2 = ['eng', 'ukr', 'kor', 'ger', 'fre', 'swe', 'chi', 'spa', 'rus', 'dut', 'ita', 'jpn', 'pol', 'por', 'vie', 'cat', 'ser', 'ind', 'cze', 'fin', 'hun', 'nor', 'rom', 'bul', 'dan', 'slo', 'cro'] #llama-2 languages
# walsdata_nllb = walsdata.loc[walsdata['wals_code'].isin(['ace','acm','acq',' aeb','af','ajp','ak','als','am','apc','ar','ars','ary','arz','as','ast','awa','ayr','azb','azj','ba','bm','ban','be','bem','bn','bho',' bjn','bo',' bs',' bug',' bg',' ca',' ceb',' cs',' cjk',' ckb',' crh',' cy',' da',' de',' dik',' dyu',' dz',' el',' en',' eo',' et',' eu',' ee',' fo',' fj',' fi',' fon',' fr',' fur',' fuv',' gaz',' gd',' ga',' gl',' gn',' gu',' ht',' ha',' he',' hi',' hne',' hr',' hu',' hy',' ig',' ilo',' id',' is',' it',' jv',' ja',' kab',' kac',' kam',' kn',' ks',' ka',' kk',' kbp',' kea',' khk',' km',' ki',' rw',' ky',' kmb',' kmr',' knc',' kg',' ko',' lo',' lij',' li',' ln ',' lt ',' lmo ',' ltg ',' lb ',' lua ',',' lg	',' ',' luo	',',' ',' lu',',' ',' ',',','v',',',' ',' mag ',',' mai	',' ',' ml ',' mar ',' min ',' mk ',' mt ',' mni ',' mos ',' mi ',' my ',' nl ',' nn ',' nb ',' npi ',' nso',' nus',' ny',' oc',' ory ',' pag ',' pa ',' pap ',' pbt ',' pes ',' plt ',' pl ',' pt ',' prs ',' quy ',' ro ',' rn ',' ru ',' sg ',' sa ',' sat ',' scn ',' shn ',' si ',' sk ',' sl ',' sm','sn',' sd','so','st',' es','sc','sr','ss','su','sv','swh	','szl','ta','taq',' tt','te','tg','tl','th','ti','tpi','tn','ts','tk','tum','tr','tw	',' tzm	',' ug','uk',' umb','ur','uzn','vec','vi',' war','wo','xh','ydd','yo','yue','zh','zsm','zu'])]
# NEED TO MAP NLLB languages to ISO-3 or WALSCODE! 

# Focus on only one family of languages? or one genus? Or variety over the world's languages?
# walsdata_indo_eur = walsdata.loc[walsdata['family'] == 'Indo-European']
# walsdata_germanic = walsdata.loc[walsdata['genus'] == 'Germanic']
# walsdata_romanic = walsdata.loc[walsdata['genus'] == 'Romanic']
# walsdata_slavic = walsdata.loc[walsdata['genus'] == 'Slavic']


#### Select relevant features

In [7]:
### feature groups
genus = ['genus']
family = ['family']
phonology = ['1A Consonant Inventories','2A Vowel Quality Inventories','3A Consonant-Vowel Ratio','4A Voicing in Plosives and Fricatives','5A Voicing and Gaps in Plosive Systems','6A Uvular Consonants','7A Glottalized Consonants','8A Lateral Consonants','9A The Velar Nasal','10A Vowel Nasalization','11A Front Rounded Vowels','12A Syllable Structure','13A Tone','14A Fixed Stress Locations','15A Weight-Sensitive Stress','16A Weight Factors in Weight-Sensitive Stress Systems','17A Rhythm Types','18A Absence of Common Consonants','19A Presence of Uncommon Consonants']
morphology = ['20A Fusion of Selected Inflectional Formatives','21A Exponence of Selected Inflectional Formatives','22A Inflectional Synthesis of the Verb','23A Locus of Marking in the Clause','24A Locus of Marking in Possessive Noun Phrases','25A Locus of Marking: Whole-language Typology','26A Prefixing vs. Suffixing in Inflectional Morphology','27A Reduplication','28A Case Syncretism','29A Syncretism in Verbal Person/Number Marking']
nominal_cat = ['30A Number of Genders','31A Sex-based and Non-sex-based Gender Systems','32A Systems of Gender Assignment','33A Coding of Nominal Plurality','34A Occurrence of Nominal Plurality','35A Plurality in Independent Personal Pronouns','36A The Associative Plural','37A Definite Articles','38A Indefinite Articles','39A Inclusive/Exclusive Distinction in Independent Pronouns','40A Inclusive/Exclusive Distinction in Verbal Inflection','41A Distance Contrasts in Demonstratives','42A Pronominal and Adnominal Demonstratives','43A Third Person Pronouns and Demonstratives','44A Gender Distinctions in Independent Personal Pronouns','45A Politeness Distinctions in Pronouns','46A Indefinite Pronouns','47A Intensifiers and Reflexive Pronouns','48A Person Marking on Adpositions','49A Number of Cases','50A Asymmetrical Case-Marking','51A Position of Case Affixes','52A Comitatives and Instrumentals','53A Ordinal Numerals','54A Distributive Numerals','55A Numeral Classifiers','56A Conjunctions and Universal Quantifiers','57A Position of Pronominal Possessive Affixes']
nominal_syn = ['58A Obligatory Possessive Inflection','59A Possessive Classification','60A Genitives, Adjectives and Relative Clauses','61A Adjectives without Nouns','62A Action Nominal Constructions','63A Noun Phrase Conjunction','64A Nominal and Verbal Conjunction']
verbal_cat = ['65A Perfective/Imperfective Aspect','66A The Past Tense','67A The Future Tense','68A The Perfect','69A Position of Tense-Aspect Affixes','70A The Morphological Imperative','71A The Prohibitive','72A Imperative-Hortative Systems','73A The Optative','74A Situational Possibility','75A Epistemic Possibility','76A Overlap between Situational and Epistemic Modal Marking','77A Semantic Distinctions of Evidentiality','78A Coding of Evidentiality','79A Suppletion According to Tense and Aspect','80A Verbal Number and Suppletion']
wordorder = ['81A Order of Subject, Object and Verb','82A Order of Subject and Verb','83A Order of Object and Verb','84A Order of Object, Oblique, and Verb','85A Order of Adposition and Noun Phrase','86A Order of Genitive and Noun','87A Order of Adjective and Noun','88A Order of Demonstrative and Noun','89A Order of Numeral and Noun','90A Order of Relative Clause and Noun','91A Order of Degree Word and Adjective','92A Position of Polar Question Particles','93A Position of Interrogative Phrases in Content Questions','94A Order of Adverbial Subordinator and Clause','95A Relationship between the Order of Object and Verb and the Order of Adposition and Noun Phrase','96A Relationship between the Order of Object and Verb and the Order of Relative Clause and Noun','97A Relationship between the Order of Object and Verb and the Order of Adjective and Noun','143A Order of Negative Morpheme and Verb','143B Obligatory Double Negation','143C Optional Double Negation','143D Optional Triple Negation','143E Preverbal Negative Morphemes','143F Postverbal Negative Morphemes','143G Minor morphological means of signaling negation','144A Position of Negative Word With Respect to Subject, Object, and Verb','144B Position of negative words relative to beginning and end of clause and with respect to adjacency to verb','144C Languages with different word order in negative clauses','144D The Position of Negative Morphemes in SVO Languages','144E Multiple Negative Constructions in SVO Languages','144F Obligatory Double Negation in SVO languages','144G Optional Double Negation in SVO languages','144H NegSVO Order','144I SNegVO Order','144J SVNegO Order','144K SVONeg Order','144L The Position of Negative Morphemes in SOV Languages','144M Multiple Negative Constructions in SOV Languages','144N Obligatory Double Negation in SOV languages','144O Optional Double Negation in SOV languages','144P NegSOV Order','144Q SNegOV Order','144R SONegV Order','144S SOVNeg Order','144T The Position of Negative Morphemes in Verb-Initial Languages','144U Double negation in verb-initial languages','144V Verb-Initial with Preverbal Negative','144W Verb-Initial with Negative that is Immediately Postverbal or between Subject and Object','144X Verb-Initial with Clause-Final Negative','144Y The Position of Negative Morphemes in Object-Initial Languages']
simple_clause = ["100A Alignment of Verbal Person Marking","101A Expression of Pronominal Subjects","102A Verbal Person Marking","103A Third Person Zero of Verbal Person Marking","104A Order of Person Markers on the Verb","105A Ditransitive Constructions: The Verb 'Give'","106A Reciprocal Constructions","107A Passive Constructions","108A Antipassive Constructions","108B Productivity of the Antipassive Construction","109A Applicative Constructions","109B Other Roles of Applied Objects","110A Periphrastic Causative Constructions","111A Nonperiphrastic Causative Constructions","112A Negative Morphemes","113A Symmetric and Asymmetric Standard Negation","114A Subtypes of Asymmetric Standard Negation","115A Negative Indefinite Pronouns and Predicate Negation","116A Polar Questions","117A Predicative Possession","118A Predicative Adjectives","119A Nominal and Locational Predication","120A Zero Copula for Predicate Nominals","121A Comparative Constructions"]
complex_sentences = ["122A Relativization on Subjects","123A Relativization on Obliques","124A 'Want' Complement Subjects","125A Purpose Clauses","126A 'When' Clauses","127A Reason Clauses","128A Utterance Complement Clauses"]
lexicon = ['129A Hand and Arm','130A Finger and Hand','131A Numeral Bases','132A Number of Non-Derived Basic Colour Categories','133A Number of Basic Colour Categories','134A Green and Blue','135A Red and Yellow','136A M-T Pronouns','137A N-M Pronouns','138A Tea']


In [8]:
### specify languages of interest
languages_mgsm = ['eng','spa','fre','swa', 'chi', 'jpn','ben','ger','rus','tel','tha']
### specify features of interest
features = genus+family+phonology+morphology+nominal_cat+nominal_syn+verbal_cat+wordorder+simple_clause+complex_sentences+lexicon

get_sim_df(languages_mgsm,features)

,eng,spa,fre,swa,chi,jpn,ben,ger,rus,tel,tha
eng,151,96,94,61,9,52,20,96,101,28,66
spa,96,147,105,60,9,47,16,80,97,24,67
fre,94,105,151,56,11,45,21,97,98,22,57
swa,61,60,56,134,11,47,20,43,63,22,56
chi,9,9,11,11,44,13,4,10,7,12,10
jpn,52,47,45,47,13,142,21,52,57,43,54
ben,20,16,21,20,4,21,53,21,24,23,17
ger,96,80,97,43,10,52,21,148,92,27,45
rus,101,97,98,63,7,57,24,92,148,29,62
tel,28,24,22,22,12,43,23,27,29,64,22
